In [21]:
import pickle
import re
from tqdm import tqdm
import binascii
import numpy as np
import random
import pandas as pd

In [2]:
with open("data/news_all.pkl", 'rb') as f:
    news = pickle.load(f)

In [3]:
news = news[:2000]

In [4]:
def get_shingles(words, n=2):
    for i in range (0,len(words)-n+1):
        yield ' '.join(words[i:i+n])

In [5]:
def clean(source):
    stop_symbols = '.,!?:;-\n\r()—'

    stop_words = ('это', 'как', 'так',
                  'и', 'в', 'над',
                  'к', 'до', 'не',
                  'на', 'но', 'за',
                  'то', 'с', 'ли',
                  'а', 'во', 'от',
                  'со', 'для', 'о',
                  'же', 'ну', 'вы',
                  'бы', 'что', 'кто',
                  'он', 'она')

    return ( [x for x in [y.strip(stop_symbols) for y in source.lower().split()] if x and (x not in stop_words)] )

In [6]:
docsAsShingleSets = {}
i = 0
for document in tqdm(news):
    shinglesInDoc  = set()
    clean_doc = clean(document['body'])
    shingles = { i for i in get_shingles(clean_doc, 5)}
    for shingle in shingles:
        crc = binascii.crc32(shingle.encode('utf-8')) & 0xffffffff
        shinglesInDoc.add(crc)
    
    docsAsShingleSets[i] = shinglesInDoc
    i += 1

100%|████████████████████████████████████| 2000/2000 [00:00<00:00, 2380.82it/s]


In [7]:
docs_num = len(news)

In [8]:
sim_matrix = np.zeros((docs_num, docs_num))

In [ ]:
for i in tqdm(range(docs_num)):
    doc_1 = docsAsShingleSets[i]
    for j in range(i+1, docs_num):
        doc_2 = docsAsShingleSets[j]
        sim_matrix[i][j] = len(doc_1.intersection(doc_2)) / len(doc_1.union(doc_2))

  0%|▏                                    | 69/20090 [00:24<1:58:47,  2.81it/s]

## MinHashing

In [9]:
numHashes = 10
maxShingleID = 2**32-1
nextPrime = 4294967311

In [10]:
def pickRandomCoeffs(k):
    randList = []
  
    while k > 0:
        randIndex = random.randint(0, maxShingleID) 
  
        while randIndex in randList:
            randIndex = random.randint(0, maxShingleID) 

        randList.append(randIndex)
        k = k - 1
    
    return randList

In [11]:
coeffA = pickRandomCoeffs(numHashes)
coeffB = pickRandomCoeffs(numHashes)

In [12]:
signatures = []

In [13]:
for i in tqdm(range(docs_num)):
    shingleIDSet = docsAsShingleSets[i]
    signature = []

    for i in range(numHashes):
        minHashCode = nextPrime + 1
        for shingleID in shingleIDSet:
            hashCode = (coeffA[i] * shingleID + coeffB[i]) % nextPrime 
            if hashCode < minHashCode:
                minHashCode = hashCode
        signature.append(minHashCode)

    signatures.append(signature)

100%|████████████████████████████████████| 2000/2000 [00:01<00:00, 1383.05it/s]


In [ ]:
numElems = int(docs_num * (docs_num - 1) / 2)
estJSim = [0 for x in range(numElems)]

In [ ]:
def getTriangleIndex(i, j):
    if i == j:
        sys.stderr.write("Can't access triangle matrix with i == j")
        sys.exit(1)
    if j < i:
        temp = i
        i = j
        j = temp

    k = int(i * (docs_num - (i + 1) / 2.0) + j - i) - 1

    return k

In [14]:
for i in tqdm(range(docs_num)):
    signature1 = signatures[i]
    for j in range(i + 1, docs_num):
        signature2 = signatures[j]

        count = 0
        for k in range(0, numHashes):
            count += signature1[k] == signature2[k]
            
#         estJSim[getTriangleIndex(i, j)] = (count / numHashes)
        sim_matrix[i][j] = (count / numHashes)
        sim_matrix[j][i] = (count / numHashes)

100%|█████████████████████████████████████| 2000/2000 [00:11<00:00, 170.97it/s]


In [15]:
threshold = 0.8

In [16]:
news[0]['body']

'Верховный суд Марий Эл 23 мая отменил решение нижестоящей инстанции о блокировке VPN-сервиса HideMy.name и направил дело на новое рассмотрение. Об этом «Медузе» сообщили представители сервиса. Сервис был заблокирован в 2018 году на основании решения одного из районных судов Йошкар-Олы. Представителей HideMy.name к процессу не привлекли; причину блокировки сотрудникам сервиса, по их словам, выяснить не удалось. «Наш прецедент доказывает, что сегодня любой сайт или бизнес в рунете можно по надуманному предлогу заблокировать практически на год. При этом снять блокировку будет возможно только после нескольких кругов бюрократического ада», — заявил глава HideMy.name Маркус Саар. На момент публикации новости страница по адресу HideMy.name по-прежнему находится в реестре запрещенных сайтов и, согласно сервису Роскомнадзора, блокируется. Сервис после блокировки располагается на другом домене — hidemyna.me. В 2017 году VPN-сервис Hideme.ru (прежнее название HideMy.name) был заблокиров

In [20]:
i = 1
for j in range(i + 1, docs_num):
    estJ = sim_matrix[i][j]
    
    if estJ > threshold:
        print(j)
        print(news[j]['body'])
        print()

12
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и стабильность». Харадинай призвал ООН уважать решение о высылке россиянина, добавив, что оно вступило в силу 31 мая. Михаил Краснощеков был задержан 28 мая в рамках операции полиции Косово, направленной, как сообщалось, против организованной преступности. Россиянин был вскоре освобожден, но ему потребовалась помощь врачей из-за травмы головы. Позже Краснощеков был переведен в больницу в Белграде.

23
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир 

Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и стабильность». Харадинай призвал ООН уважать решение о высылке россиянина, добавив, что оно вступило в силу 31 мая. Михаил Краснощеков был задержан 28 мая в рамках операции полиции Косово, направленной, как сообщалось, против организованной преступности. Россиянин был вскоре освобожден, но ему потребовалась помощь врачей из-за травмы головы. Позже Краснощеков был переведен в больницу в Белграде.

144
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и 

Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и стабильность». Харадинай призвал ООН уважать решение о высылке россиянина, добавив, что оно вступило в силу 31 мая. Михаил Краснощеков был задержан 28 мая в рамках операции полиции Косово, направленной, как сообщалось, против организованной преступности. Россиянин был вскоре освобожден, но ему потребовалась помощь врачей из-за травмы головы. Позже Краснощеков был переведен в больницу в Белграде.

265
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и 

Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и стабильность». Харадинай призвал ООН уважать решение о высылке россиянина, добавив, что оно вступило в силу 31 мая. Михаил Краснощеков был задержан 28 мая в рамках операции полиции Косово, направленной, как сообщалось, против организованной преступности. Россиянин был вскоре освобожден, но ему потребовалась помощь врачей из-за травмы головы. Позже Краснощеков был переведен в больницу в Белграде.

386
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и 

Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и стабильность». Харадинай призвал ООН уважать решение о высылке россиянина, добавив, что оно вступило в силу 31 мая. Михаил Краснощеков был задержан 28 мая в рамках операции полиции Косово, направленной, как сообщалось, против организованной преступности. Россиянин был вскоре освобожден, но ему потребовалась помощь врачей из-за травмы головы. Позже Краснощеков был переведен в больницу в Белграде.

507
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и 

...

Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и стабильность». Харадинай призвал ООН уважать решение о высылке россиянина, добавив, что оно вступило в силу 31 мая. Михаил Краснощеков был задержан 28 мая в рамках операции полиции Косово, направленной, как сообщалось, против организованной преступности. Россиянин был вскоре освобожден, но ему потребовалась помощь врачей из-за травмы головы. Позже Краснощеков был переведен в больницу в Белграде.

1475
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и

Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и стабильность». Харадинай призвал ООН уважать решение о высылке россиянина, добавив, что оно вступило в силу 31 мая. Михаил Краснощеков был задержан 28 мая в рамках операции полиции Косово, направленной, как сообщалось, против организованной преступности. Россиянин был вскоре освобожден, но ему потребовалась помощь врачей из-за травмы головы. Позже Краснощеков был переведен в больницу в Белграде.

1596
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и

Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и стабильность». Харадинай призвал ООН уважать решение о высылке россиянина, добавив, что оно вступило в силу 31 мая. Михаил Краснощеков был задержан 28 мая в рамках операции полиции Косово, направленной, как сообщалось, против организованной преступности. Россиянин был вскоре освобожден, но ему потребовалась помощь врачей из-за травмы головы. Позже Краснощеков был переведен в больницу в Белграде.

1717
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и

Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и стабильность». Харадинай призвал ООН уважать решение о высылке россиянина, добавив, что оно вступило в силу 31 мая. Михаил Краснощеков был задержан 28 мая в рамках операции полиции Косово, направленной, как сообщалось, против организованной преступности. Россиянин был вскоре освобожден, но ему потребовалась помощь врачей из-за травмы головы. Позже Краснощеков был переведен в больницу в Белграде.

1838
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и

Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и стабильность». Харадинай призвал ООН уважать решение о высылке россиянина, добавив, что оно вступило в силу 31 мая. Михаил Краснощеков был задержан 28 мая в рамках операции полиции Косово, направленной, как сообщалось, против организованной преступности. Россиянин был вскоре освобожден, но ему потребовалась помощь врачей из-за травмы головы. Позже Краснощеков был переведен в больницу в Белграде.

1959
Россиянин Михаил Краснощеков, работающий в миссии ООН в Косово, объявлен персоной нон грата. Об этом написал в фейсбуке премьер-министр Косово Рамуш Харадинай. Краснощеков, заявил премьер-министр, «занимался деятельностью, направленной против конституционного строя Косово, расшатывал мир и

In [27]:
test = pd.DataFrame(news)

In [24]:
test.head()

,author,body,date,title
0,Meduza,Верховный суд Марий Эл 23 мая отменил решени...,2019-05-31 15:04:00,VPN-сервис HideMy.name выиграл суд о блокировке
1,Рамуш Харадинай,"Россиянин Михаил Краснощеков, работающий в ми...",2019-05-31 12:46:00,Власти Косово объявили российского сотрудника...
2,Коммерсант,Министерство образования и науки РФ ответило н...,2019-05-31 14:03:00,В Минобрнауки назвали легитимным формирование ...
3,"""Красная звезда""","В главном храме Минобороны, который строят в ...",2019-05-31 13:57:00,В главном храме Минобороны изобразят подвиг 28...
4,Forbes,"Следователи, ведущие дело «Седьмой студии», в...",2019-05-31 12:22:00,В деле «Седьмой студии» появится новый подоз...


In [25]:
test.drop_duplicates('body', inplace=True)

In [28]:
test.shape

(2000, 4)